In [2]:
#Librerias

import google.generativeai as genai
import getpass
import json
import os
import time
import base64
from google.colab import userdata

In [3]:
#Configuracion de API

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
#Base de datos

kyokushin_data = {
    "blanco": {
        "terminologia": "Osu (Saludo y respeto), Kyokushin (Verdad máxima), Sensei (Instructor), Sempai (Estudiante mayor).",
        "tecnicas_basicas": ["Zenkutsu Dachi", "Gedan Barai", "Oi Tsuki", "Mae Geri"],
        "katas": "Taikyoku Sono Ichi.",
        "requisitos": "Para obtener el cinturón naranjo, debes demostrar dominio de las posturas básicas, los bloqueos, los puños fundamentales, la patada Mae Geri y la Kata Taikyoku Sono Ichi. También se evalúa la comprensión de la terminología básica."
    },
    "naranjo": {
        "terminologia": "Chudan (Zona media), Jodan (Zona alta), Gedan (Zona baja), Yame (Parar).",
        "tecnicas_basicas": ["Seiken Jodan Gyaku Tsuki", "Shuto Gammen Uchi", "Hiza Geri"],
        "katas": "Taikyoku Sono Ichi, Ni.",
        "requisitos": "Para obtener el cinturón azul, se requiere dominar las técnicas de los cinturones blanco y naranjo. Se evalúa el Kihon, las Katas Taikyoku Sono Ichi y Ni, y Jiyu Kumite (combate libre) con control y espíritu de lucha."
    },
    "azul": {
        "terminologia": "Kamae (Postura de guardia), Shihan (Maestro), Ki (Energía interior).",
        "tecnicas_basicas": ["Tate Tsuki", "Ura Mawashi Geri", "Jodan Mawashi Geri"],
        "katas": "Taikyoku Sono Ichi, Ni, San.",
        "requisitos": "Para obtener el cinturón amarillo, se debe dominar el Kihon avanzado, las Katas Taikyoku de Ichi a Tres y Jiyu Kumite con mayor intensidad y un mejor control de las técnicas."
    },
    "amarillo": {
        "terminologia": "Kiai (Grito de energía), Renraku (Combinación), Kumite (Combate).",
        "tecnicas_basicas": ["Mawashi Tsuki", "Ushiro Mawashi Geri"],
        "katas": "Pinan Sono Ichi, Ni.",
        "requisitos": "Para obtener el cinturón verde, se espera el dominio de las Katas Taikyoku de Ichi a Tres, las Katas Pinan de Ichi a Dos, y el examen de graduación consiste en un Kihon más complejo, combinaciones (Renraku), y Jiyu Kumite contra varios oponentes, demostrando resistencia y espíritu."
    },
    "verde": {
        "terminologia": "Naka (Interior), Soto (Exterior), Jyu Kumite (Combate libre).",
        "tecnicas_basicas": ["Morote Tsuki", "Oroshi Uchi Ude Uke", "Kakato Geri"],
        "katas": "Pinan Sono Ichi, Ni, San.",
        "requisitos": "Para obtener el cinturón marrón, debes dominar las Katas Taikyoku de Ichi a Tres y las Katas Pinan de Ichi a Tres. El examen de graduación incluye un Kihon muy avanzado y una práctica de Kumite más rigurosa y con más rondas."
    }
}

In [5]:
#Establecer interaccion y prompt

def get_chat_response(user_query, user_belt):
    """
    Genera una respuesta de texto utilizando un prompt optimizado.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    context_data = kyokushin_data.get(user_belt.lower(), {})

    if not context_data:
        return "Lo siento, la información para ese cinturón no está disponible en mi base de datos."

    prompt = f"""
    Eres un asistente personal de karate Kyokushin para estudiantes principiantes.
    Actúa como un Sempai (estudiante mayor) servicial y conocedor.
    Responde a la pregunta del usuario de forma concisa, amigable y clara, utilizando los datos proporcionados a continuación.
    Si la respuesta a la pregunta del usuario no se encuentra en el contexto, simplemente indica que no tienes esa información.
    NO intentes inventar la respuesta.

    --- Contexto del Cinturón '{user_belt.capitalize()}' ---
    {json.dumps(context_data, indent=2)}

    --- Pregunta del Usuario ---
    {user_query}

    Respuesta:
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Ocurrió un error al contactar con la IA: {e}"

In [6]:
def generate_kyokushin_image(image_prompt):
    """
    Genera una imagen de Kihon utilizando la API de Google Gemini y la guarda en un archivo.
    """
    try:
        print("Enviando solicitud de generación de imagen a Gemini...")
        model = genai.GenerativeModel('gemini-2.5-flash-image-preview')

        # El prompt ahora es una solicitud de imagen específica para el modelo multimodal
        prompt = f"Genera una imagen de una persona haciendo {image_prompt} en un dojo de karate Kyokushin. La persona es un experto practicante de Kyokushin, con un karategi blanco y un cinturón negro. La pose es de acción, demostrando la técnica de forma correcta y potente. Estilo fotográfico, con buena iluminación y enfoque."

        response = model.generate_content(prompt)

        # Se extrae el contenido de la imagen de la respuesta
        if response and response.candidates and response.candidates[0].content and response.candidates[0].content.parts:
            # Encuentra la parte que contiene los datos en línea (inlineData)
            image_part = next((part for part in response.candidates[0].content.parts if part.inline_data), None)

            if image_part:
                image_data = image_part.inline_data.data
                image_bytes = base64.b64decode(image_data)

                filename = f"kihon_{image_prompt.replace(' ', '_').replace('/', '')}.jpeg"
                with open(filename, "wb") as f:
                    f.write(image_bytes)
                return f"Imagen generada y guardada como: {filename}"
            else:
                return "La respuesta de la API no contiene datos de imagen. Intenta con un prompt diferente."
        else:
            return "No se pudo generar la imagen. La respuesta de la API está incompleta."

    except Exception as e:
        return f"Ocurrió un error al generar la imagen con Gemini: {e}"

# --- Bucle de Interacción del Usuario ---

if __name__ == "__main__":
    print("--- Asistente de IA de Kyokushin (Script de Python) ---")

    while True:
        print("\n¿Qué deseas hacer?")
        print("1. Chatear con el Sempai (Asistente de Texto)")
        print("2. Generar una imagen de Kihon")
        print("3. Salir")

        choice = input("Ingresa tu opción (1, 2, o 3): ")

        if choice == '1':
            user_belt = input("¿Cuál es tu cinturón (blanco, naranjo, azul, amarillo, verde)? ").strip()
            user_question = input("¿En qué puedo ayudarte? ").strip()

            if user_belt and user_question:
                print("\nConsultando al Sempai...")
                response = get_chat_response(user_question, user_belt)
                print("---")
                print("Respuesta:", response)
                print("---")
            else:
                print("Por favor, ingresa tu cinturón y una pregunta válida.")

        elif choice == '2':
            # Se crea una lista unificada de todos los kihon disponibles
            all_kihon = []
            for belt in ["blanco", "naranjo", "azul", "amarillo", "verde"]:
                all_kihon.extend(kyokushin_data[belt]["tecnicas_basicas"])

            print("\nListado de Kihones disponibles para generar una imagen:")
            for i, kihon in enumerate(all_kihon):
                print(f"{i + 1}. {kihon}")

            try:
                kihon_choice = int(input("\nElige el número de la técnica que quieres generar: "))
                if 1 <= kihon_choice <= len(all_kihon):
                    selected_kihon = all_kihon[kihon_choice - 1]
                    print(f"\nGenerando imagen para '{selected_kihon}'. Esto puede tomar un momento...")
                    response = generate_kyokushin_image(selected_kihon)
                    print("---")
                    print(response)
                    print("---")
                else:
                    print("Opción no válida. Por favor, elige un número de la lista.")
            except ValueError:
                print("Entrada no válida. Por favor, ingresa un número.")

        elif choice == '3':
            print("¡Osu! ¡Hasta la próxima!")
            break

        else:
            print("Opción no válida. Por favor, elige 1, 2 o 3.")


--- Asistente de IA de Kyokushin (Script de Python) ---

¿Qué deseas hacer?
1. Chatear con el Sempai (Asistente de Texto)
2. Generar una imagen de Kihon
3. Salir
Ingresa tu opción (1, 2, o 3): 2

Listado de Kihones disponibles para generar una imagen:
1. Zenkutsu Dachi
2. Gedan Barai
3. Oi Tsuki
4. Mae Geri
5. Seiken Jodan Gyaku Tsuki
6. Shuto Gammen Uchi
7. Hiza Geri
8. Tate Tsuki
9. Ura Mawashi Geri
10. Jodan Mawashi Geri
11. Mawashi Tsuki
12. Ushiro Mawashi Geri
13. Morote Tsuki
14. Oroshi Uchi Ude Uke
15. Kakato Geri

Elige el número de la técnica que quieres generar: 4

Generando imagen para 'Mae Geri'. Esto puede tomar un momento...
Enviando solicitud de generación de imagen a Gemini...


---
Ocurrió un error al generar la imagen con Gemini: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-image-preview:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
---

¿Qué deseas hacer?
1. Chatear con el Sempai (Asistente de Texto)
2. Generar una imagen de Kihon
3. Salir
Ingresa tu opción (1, 2, o 3): 1
¿Cuál es tu cinturón (blanco, naranjo, azul, amarillo, verde)? azul
¿En qué puedo ayudarte? Que movimientos deberia aprender

Consultando al Sempai...
---
Respuesta: ¡Hola!  Como principiante de Kyokushin,  para tu nivel de cinturón azul deberías enfocarte en perfeccionar las técnicas básicas: Tate Tsuki, Ura Mawashi Geri, y Jodan Mawashi Geri.  También deberías practicar los Katas Taikyoku Sono Ichi, Ni, y San. ¡Oss!

---

¿Qué deseas hacer?
1. Chatear con el Sempai (Asistente 